### Dream Team
* Batsman:     SD Chitnis (batting order - 1, SR = 250)<br>
* Batsman:     Gurkeerat Singh (batting order - 2, SR = 241) <br>
* Batsman:     AUK Pathan (batting order - 3, SR = 233.3) <br>
* all-rounder: SM Curran (batting order - 4, SR = 283.33, Wickets per 100 games = 5.29) <br>
* bowler:      MJ McClenaghan (batting order - 5, SR = 222.22, Wickets per 100 games = 5.57) <br>
* wk:          Kamran Akmal (batting order - 6, SR = 300) <br>
* all-rounder: JR Hopes (batting order - 7, SR = 325, Wickets per 100 games = 3.79) <br>
* all-rounder: KK Cooper (batting order - 6, SR = 220, Wickets per 100 games = 6.0) <br>
* all-rounder: CH Morris (batting order - 7, SR = 242.85, Wickets per 100 games = 5.74) <br>
* bowler:      Umar Gul (batting order - 8, SR = 218.18 ,Wickets per 100 games = 9.15) <br>
* bowler:      WPUJC Vaas (batting order - 9, SR = 171.42, Wickets per 100 games = 7.63) <br>
* captain: JR Hopes <br>
* vice-captain: Kamran Akmal <br> <br>
I chose 3 batsman, 4 all-rounders, 3 bowlers, and 1 wicket keeper primarily based on their highest strike rate depending on their batting order. The biggest part of the analysis was to find the best players using trial and error. The queries written below helped me in the process of finding batsmen with high strike rate, bowlers with most number of wickets per one hundred matches and all-rounders with both good strike rates and wickets. One of the reasons I chose wickets per 100 games over bowler economy rate is to maximise opponent's risk of losing a wicket while facing the bowlers of my team. It has been proven that the higher the wicket/100 games the less would be the economy rate.   

In [1]:
# Libraries for data manipulation and linear algebra
import numpy as np
import pandas as pd

In [2]:
# Reading the .csv file as a dataframe
dataset = pd.read_csv("IPL.csv") 

In [3]:
# The following script is almost exactly similar as the script used in problem 1. The only difference is that 
#     this function runs on every single player as opposed to only Mumbai Indians players. 
sort_game = dataset[['id','inning','over','ball','batsman_runs']].sort_values(by = ['id','inning','over','ball'])
# create a dictionary to keep track of players and their inning/over/ball
player_name = {}
counter = 1
# for every game in the dataframe 
for i in sort_game['id'].unique():
    # every numer of innings (1 and 2)
    for j in range(1,3):
        # get the columns inning, over and ball for a specific game and inning 
        sort_game = dataset.query("id == @i and inning == @j").sort_values(by = ['inning','over','ball'])
        # for all games 
        for x in range(len(sort_game.index)): 
            # if inning, ball, over not already present for a player, add this data to the dictionary 
            if (i, j, sort_game.batsman.values[x]) not in player_name: 
                player_name.update({(i,j,sort_game.batsman.values[x]):counter}) 
                counter += 1 
        # reset counter 
        counter = 1 
        
# prepare to map the dictionary in the dataframe        
dataset['temp'] = tuple(zip(dataset.id.values, dataset.inning.values, dataset.batsman.values))
# map the dictionary 
dataset['batting_order'] = dataset.temp.map(player_name)
dataset = dataset.drop('temp',axis=1) 
# create a dataframe with the columns we need 
sort_game = dataset[np.hstack((dataset.columns.values[:6], dataset.columns.values[-1], dataset.columns.values[6:-1]))]
# sort the columns of the dataframe on ascending order 
sort_game = sort_game.sort_values(by=['inning','over','ball'])
# calculate the total runs made by batsmman
total_runs = dataset.groupby('batsman')['batsman_runs'].sum().sort_values(ascending=False)
player_list = total_runs.index 
total_runs = dataset.query("batsman in @player_list").groupby(['batsman','batting_order'])['batsman_runs'].sum().reset_index()
total_balls = dataset.query("batsman in @player_list").groupby(['batsman','batting_order'])['id'].count().sort_values(ascending=False).reset_index()
get_strike_rate = total_runs.merge(total_balls, on=['batsman','batting_order'], how = 'outer') 
# calculate the strike rate 
get_strike_rate['strike_rate'] = ((get_strike_rate.batsman_runs/get_strike_rate.id)*100) 
get_strike_rate = get_strike_rate.drop(['id','batsman_runs'], axis=1)  

In [4]:
# query depending on strike rate (We can set it to 150, 200, 250 or above to find players with highest SR) 
batsman_info = get_strike_rate[get_strike_rate['strike_rate'] > 200][['batsman','batting_order','strike_rate']].sort_values(by = ['strike_rate'], ascending = False)

In [5]:
# I found batsmen with the highest strike rate depending on their batting orddr by setting up the number 
#   in the query below.
batsman_info[batsman_info['batting_order'] == 4]

,batsman,batting_order,strike_rate
1148,P Negi,4,400.000000
1533,SM Curran,4,283.333333
975,MK Lomror,4,266.666667
970,MJ McClenaghan,4,222.222222


In [6]:
# write a function that calculates total wicket taken and matches played by bowlers

# a list that stores data of bowls/id/overs where a wicket was taken
bowler_data = dataset[dataset['is_wicket'] == 1][['bowler','dismissal_kind','id','bowling_team','is_wicket']]
# a list that stores data of bowls/id/overs where a wicket was not taken 
bowler_data2 = dataset[dataset['is_wicket'] == 0][['bowler','dismissal_kind','id','bowling_team','is_wicket']]

def calc_wicket(bowler_data, bowler_data2):
    
    # get a list of all bowlers name 
    bowlers_name = bowler_data['bowler'].unique()
    data = []
    
    # for every bowler in the list
    for bowler in bowlers_name:
        
        # calculate the total number of wickets taken by that bowler
        total_wickets = bowler_data[bowler_data['bowler'] == bowler]['is_wicket'].sum()
        # calculate the total number of games played by that bowler 
        total_games = ((bowler_data[bowler_data['bowler'] == bowler]['id']).count())+((bowler_data2[bowler_data2['bowler'] == bowler]['id']).count())
        # add the two values to the list 
        data.append([bowler,total_wickets, total_games])
    
    outputdf = pd.DataFrame(data, columns = ['bowler','total_wickets','total_games'])
    
    return outputdf

bowler_info = calc_wicket(bowler_data, bowler_data2) 
bowler_info = bowler_info.sort_values(by=['total_wickets'], ascending=False)
# calculate total number of wicket taken by a bowler per 100 runs 
bowler_info['wickets_per_100games'] = (bowler_info['total_wickets']/bowler_info['total_games'])*100
bowler_info = bowler_info.sort_values(by = ['wickets_per_100games'], ascending = False)

In [7]:
# only consider bowlers who have played more then 150 games to a more accurate data on wickets/100 games
bowler_info = bowler_info[bowler_info['total_games'] >= 200][['bowler','total_wickets','total_games','wickets_per_100games']]

In [8]:
bowler_info.head() 

,bowler,total_wickets,total_games,wickets_per_100games
60,Sohail Tanvir,24,265,9.056604
326,L Ngidi,23,267,8.614232
306,K Rabada,66,840,7.857143
29,WPUJC Vaas,22,288,7.638889
56,CRD Fernando,19,249,7.630522


In [9]:
# candidates who would be an all-rounder depending in their batting order 
# all rounder with high strike rate depending on batting order 1,2,..,8
ar_high_sr_b1 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 1)]['batsman'])
ar_high_sr_b2 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 2)]['batsman'])
ar_high_sr_b3 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 3)]['batsman'])
ar_high_sr_b4 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 4)]['batsman'])
ar_high_sr_b5 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 5)]['batsman'])
ar_high_sr_b6 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 6)]['batsman'])
ar_high_sr_b7 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 7)]['batsman'])
ar_high_sr_b8 = list(batsman_info[(batsman_info['strike_rate'] >= 150) & (batsman_info['batting_order'] == 8)]['batsman'])

In [10]:
# gets players with high strike rate 
arr1 = np.array(ar_high_sr_b6) 
# get players with high number of wickets 
arr2 = np.array(bowler_info['bowler'].apply(str))

In [11]:
# merge arr1 and arr2 to get players with both high strike rate and high number of wickets 
common_n = np.intersect1d(arr1,arr2)
print(common_n) 

['K Gowtham' 'KK Cooper']


In [12]:
# for every all-rounder (name_list), check their stats with top 3 batsman and top 3 bowlers,
# some examples shown below
# Through a process of trial and error with the help of queries in this script
#     find the best batsman, bowlers, and all-rounders to win fantasy cricket.

In [13]:
# Query that shows a players strike rate (I used it with all of the players I chose to make sure they had high SR
#     for their batting order)
batsman_info[batsman_info['batsman'] == 'AUK Pathan'] 

,batsman,batting_order,strike_rate
188,AUK Pathan,3,233.333333


In [14]:
# Replace JR Hopes with other bowlers/all-rounders in the team to get their wicket rate 
bowler_info[bowler_info['bowler'] == 'JR Hopes']

,bowler,total_wickets,total_games,wickets_per_100games
11,JR Hopes,14,369,3.794038
